# Recommendation Based on Patient's Complaint

Sequence to sequence task

In [2]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.0/353.0 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.5/283.5 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: torchao
    Found existing installation: torchao 0.10.0
    Uninstalling torchao-0.10.0:
      S

## Imports

In [1]:
import pandas as pd
from unsloth import FastLanguageModel
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, Trainer, TrainingArguments, BitsAndBytesConfig
import transformers
import torch
import torch.nn as nn
from peft import LoraConfig, get_peft_model, PeftModel
import os
from trl import SFTTrainer
os.environ["WANDB_DISABLED"] = "true"

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## Load Data

In [17]:
df = pd.read_csv('https://drive.google.com/uc?id=1eX3v3E7YR-faYm7tXWzCsZH998oOsIqC', encoding="latin-1")
ds = Dataset.from_pandas(df)
print(df.head(10))

                                    Symptoms_comment  \
0  I am dealing with Fever, Cough recently, and I...   
1  These past few days, Headache, Fatigue have be...   
2  These past few days, Shortness of breath have ...   
3  Lately, I've noticed symptoms like Nausea, Vom...   
4  Lately, I've noticed symptoms like Sore Throat...   
5  I am dealing with Joint Pain, Fatigue recently...   
6  I've been experiencing Chest Pain, Dizziness, ...   
7  I started feeling Itching, Redness, and it doe...   
8  I started feeling Abdominal Pain, Bloating, an...   
9  I am dealing with Fatigue, Sadness recently, a...   

                                  Causes_and_Disease  \
0  The symptoms suggest Viral Infection, and the ...   
1  These signs are commonly associated with Stres...   
2  Based on the symptoms, the likely cause is Pol...   
3  These signs are commonly associated with Food ...   
4  These signs are commonly associated with Bacte...   
5  Based on the symptoms, the likely cause is R

## Load Model

In [23]:
MAX_SEQ = 1024
model, tokenizer = FastLanguageModel.from_pretrained(
    "unsloth/Llama-3.2-1B-Instruct",
    max_seq_length=MAX_SEQ,
    load_in_4bit=True,
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

==((====))==  Unsloth 2025.11.3: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


## Set Up LoRA

In [26]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

## Format Data

In [27]:
alpaca_prompt = """Based on the patient's symptoms, cause/disease, and the medical treatment guidelines below, give a short recommendation to the patient (3 sentences max).

### Symptoms:
{}

### Cause or Disease:
{}

### Medical Treatment Guidelines:
{}

### Recommendation:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_func(examples):
    symptoms = examples["Symptoms_comment"]
    causes_or_diseases = examples["Causes_and_Disease"]
    fetched_recommendations = examples["Fetched_recommendation"]
    recommendations = examples["Medicine_recommendation"]
    texts = []
    for symptom, cause_or_disease, fetched_recommendation, recommendation in zip(symptoms, causes_or_diseases, fetched_recommendations, recommendations):
        text = alpaca_prompt.format(symptom, cause_or_disease, fetched_recommendation, recommendation) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

dataset = ds.map(formatting_func, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

## Fine-tune Model

In [29]:
training_args = TrainingArguments(
    output_dir="indoqa-lora-1b",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=50,
    num_train_epochs=5,
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=20,
    save_steps=400,
    save_total_limit=2,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    seed=42,
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ,
    packing=False,
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False,
    },
)

trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/100 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100 | Num Epochs = 5 | Total steps = 65
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 11,272,192 of 1,247,086,592 (0.90% trained)


Step,Training Loss
20,1.192900
40,0.758600
60,0.328100


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


TrainOutput(global_step=65, training_loss=0.7214958466016329, metrics={'train_runtime': 192.0129, 'train_samples_per_second': 2.604, 'train_steps_per_second': 0.339, 'total_flos': 2232672526491648.0, 'train_loss': 0.7214958466016329, 'epoch': 5.0})

## Save Fine-tuned Model

In [30]:
model.save_pretrained("llama-recommendation-fine-tuned")
tokenizer.save_pretrained("llama-recommendation-fine-tuned")

import shutil
from google.colab import files

folder = "llama-recommendation-fine-tuned"
shutil.make_archive(folder, 'zip', folder)
files.download(folder + ".zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# INFERENCE

In [31]:
MAX_SEQ = 1024
adapter_path = "llama-recommendation-fine-tuned"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=adapter_path,
    max_seq_length=MAX_SEQ,
    dtype=None,
    load_in_4bit=True,
)

FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.11.3: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (

In [34]:
symptom = ""
cause_or_disease = ""
fetched_recommendation = ""

symptom = "I am dealing with Joint Pain, Fatigue recently, and It's worrying me a bit."
cause_or_disease = "Based on the symptoms, the likely cause is Rheumatoid Arthritis, which typically leads to Arthritis."
fetched_recommendation = """
Disease: CHIKUNGUNYA
Treatment: treatment
ï‚· Mild exercises and physiotherapy may be suggested in recovering persons.
ï‚· Exposure to warm environment may be suggested
ï‚· Non weight bearing exercises
ï‚· Surgery in severely damaged joints
Pharmacological treatment
ï‚· Acute stage of illness
ï‚· Treat symptomatically (Tab. paracetamol 1 gm 3-4 times a day for fever, headache and
pain.)
ï‚· Avoid aspirin and steroid because of risk of GI side effects and Reyeâ€™s syndrome with
aspirin.
ï‚· Tab. hydroxychloroquinine 200mg orally once daily or Tab. chloroquinine phosphate
300mg per day for a period of 4 weeks in cases where arthralgia is refractory to other
drugs.
ï‚· Sideeffects of hydroxychloroquinine include retinal damage andelevated liver enzymes.
26
Common Conditions
"""

prompt = alpaca_prompt.format(symptom, cause_or_disease, fetched_recommendation, "")

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=80,
        temperature=0.1,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

if "### Recommendation:" in response:
    answer = response.split("### Recommendation:")[-1].strip()
else:
    answer = response.strip()

print(f"\nTest:")
print(f"Symptom: {symptom}")
print(f"Cause or disease: {cause_or_disease}")
print(f"Answer: {answer}")
print("-" * 60)


Test:
Symptom: I am dealing with Joint Pain, Fatigue recently, and It's worrying me a bit.
Cause or disease: Based on the symptoms, the likely cause is Rheumatoid Arthritis, which typically leads to Arthritis.
Answer: Acute joint pain and fatigue should be managed symptomatically with Paracetamol (1 gm 3-4 times daily) and avoiding aspirin and steroids. For chronic joint inflammation, Hydroxychloroquine (200mg orally once daily) is a preferred option, though side effects include retinal damage.
------------------------------------------------------------
